# 18.06 pset 8 (due *Friday* November 3 at 11am)

## Problem 1

**(a)** The following Julia code constructs 20 random orthogonal matrices $Q$ (a square matrix with orthonormal columns) and prints their determinants.  Try running it a few times.  **Explain what you observe** using the properties of determinants:

(Hint: $Q^T = Q^{-1}$.).

In [ ]:
m = 5  # you can try changing this too if you want

for i = 1:20
    A = randn(m,m) # a random m×m matrix
    Q, R = qr(A)   # QR factorizing A gives a random Q
    Q = Q * diagm(rand([-1,+1],m)) # choose signs randomly too

    println(round(det(Q), 13)) # print determinant, rounded to 13 digits
end

**(b)** The following code generates 20 random 5×5 "anti-symmetric" (or "skew-symmetric") matrices, and prints their determinants.  This is a square matrix $A$ with $A^T = -A$.  **Explain what you observe** using the properties of determinants.

**(c)** What happens if you try the anti-symmetric experiment again, but change the size (`m`) to 6?  Why?

In [ ]:
m = 5  # you can try changing this too if you want

for i = 1:20
    A = randn(m,m) # a random m×m matrix
    A = A - A' # make it skew-symmetric by subtracting its transpose

    println(round(det(A), 13)) # print determinant, rounded to 13 digits
end

## Problem 2

**(a)** Find the eigenvalues and eigenvectors of
$$
A = \begin{pmatrix} 1 & 4 \\ 2 & 3 \end{pmatrix}, \qquad A + 2I = \begin{pmatrix} 3 & 4 \\ 2 & 5 \end{pmatrix}
$$

**(b)** If $Ax=\lambda x$, then what is $(A+\alpha I)x$?   Therefore, how are the eigenvalues and eigenvectors of $A+\alpha I$ related to those of $A$?   Is this consistent with your answer from (a)?

## Problem 3

(From Strang, section 6.1, problem 11.)

Here is a strange fact about 2×2 matrices with eigenvalues $\lambda_1 \ne \lambda_2$: the columns of $A - \lambda_1 I$ are multiples of the eigenvector $x_2$.  *(You can see this if you look at the first 2×2 example from the [lecture-20 notebook](http://nbviewer.jupyter.org/github/stevengj/1806/blob/spring17/lectures/Eigenvalue-Polynomials.ipynb#Eigenvectors).)*   **Why?**

(Hint: think about the column and null spaces of $A - \lambda_1 I$.)

## Problem 4

In class, we saw that taking a random vector $x$ and repeatedly multiplying by $A$ (i.e. computing $A^n x$ for a large $n$) quickly gives something proportional to the *eigenvector with the largest |λ|*.  This happens for any $m\times m$ matrix with $m$ eigenvalues of distinct magnitudes.

**(a)** How could you modify this process to instead obtain the *eigenvector with the smallest |λ|*?

You can assume that $A$ is nonsingular.  If $A$ is singular, why is there an easy way to find the eigenvector with the smallest |λ|?

**(b)** The code below applies the repeated-multiplication process to the [2×2 example from lecture 20](http://nbviewer.jupyter.org/github/stevengj/1806/blob/spring17/lectures/Eigenvalue-Polynomials.ipynb#Eigenvalue-example:).   Run it to see it converging to the largest-|λ| eigenvector.  Then modify the code to your answer from (a) and run it again to verify that it converges to the smallest-|λ| eigenvector.

In [ ]:
A = [ 1 1 
     -2 4]
eigvals(A)

In [ ]:
x = [-17, 6] # a random vector
for i = 1:20
    x = A*x
    x = x / x[1] # normalize it to make x[1]=1, for comparison with lecture
    println(x)
end

It converges to (1,2), the eigenvector for λ=3.  Your modified code should converge instead to (1,1), the eigenvector for λ=2.